<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-de-aprendizaje-karelcuevas/blob/main/SVD_Equipo58.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Actividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Maximiliano Morones Gómez-A01793815
*   Jesus Esteiner Alonso Moreno-A01793554
*   Karel Alejandro Cuevas Hernández-A00301492
*   Kevin Josue Molina Robles-A01793687


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.


#**Ejercicio-1.** 

1. Descarga los archivos

In [ ]:
import requests, zipfile
from io import BytesIO

import pandas as pd

# Datos_de_calidad_del_agua_2020/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv
url='https://archive.ics.uci.edu/ml/machine-learning-databases/00232/RCdata.zip'
req=requests.get(url)

# Descomprimir el archivo
zipfile.ZipFile(BytesIO(req.content)).extractall('zip/')

# Generar el DataFrame 
df_geoplaces=pd.read_csv("zip/geoplaces2.csv", encoding = 'latin-1')
df_rating = pd.read_csv('zip/rating_final.csv', encoding = 'latin-1')

Al revisar la info de los dos DataFrame, podemos verificar que ninguno de los dos tiene valores NULOS, por lo que podemos proseguir.


In [ ]:
df_geoplaces.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   placeID         130 non-null    int64  
 1   latitude        130 non-null    float64
 2   longitude       130 non-null    float64
 3   the_geom_meter  130 non-null    object 
 4   name            130 non-null    object 
 5   address         130 non-null    object 
 6   city            130 non-null    object 
 7   state           130 non-null    object 
 8   country         130 non-null    object 
 9   fax             130 non-null    object 
 10  zip             130 non-null    object 
 11  alcohol         130 non-null    object 
 12  smoking_area    130 non-null    object 
 13  dress_code      130 non-null    object 
 14  accessibility   130 non-null    object 
 15  price           130 non-null    object 
 16  url             130 non-null    object 
 17  Rambience       130 non-null    obj

In [ ]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userID          1161 non-null   object
 1   placeID         1161 non-null   int64 
 2   rating          1161 non-null   int64 
 3   food_rating     1161 non-null   int64 
 4   service_rating  1161 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 45.5+ KB


In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

# función para poder hacer los llamados de las diferentes opciones de evaluación.
def T_SVD(variables_fil, variables_col, values='rating', index='userID', columns='name', nComp=None, nCompRedu=None, me_gusta=None):
  
  # Separados ya que ejecutaremos esta función varias veces seguidas y queremos identificar los datos.
  print("\n\n######################## Variables recibidas en este ejercicio inicio ##################################")
  print("Variables Filas        : ", ", ".join(variables_fil))
  print("Variables Columnas     : ", ", ".join(variables_col))
  print("Analizando variable    : ", values)
  print("Indexado por           : ", index)
  print("Columnas               : ", columns)
  print("Total Componentes      : ", nComp)
  print("Componentes Reducidos  : ", nCompRedu)
  print("Elemento buscado       : ", me_gusta)
  print("######################## Variables recibidas en este ejercicio fin ##################################")
  print("")
  print("*** RESULTADOS  ")

  # TABLA RELACIONAL
  tabla_relacional = pd.merge(variables_fil , variables_col , on='placeID')
  print("\nTamaño de la matriz: ", tabla_relacional.shape)
  tabla_relacional.head(10)

  # TRASPUESTA MATRIZ
  X_P=tabla_relacional.pivot_table(values=values, index=index, columns=columns, fill_value=0)
  X_P.head(10)
  print('Total de elementos de la matriz de utilidad: %d' % (X_P.size))
  print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(X_P)))
  print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(X_P) / X_P.size))

  # Generamos la transpuesta del PIVOT
  X = X_P.T

  # Obtengamos la cantidad de información contenida en la factorización SVD, considerando 
  # los valores singulares más representativos de dicha factorización.
  # Para este caso inicial usaremos todos los vectores singulares, para luego 
  # compararlo con el caso al reducir su dimensión.

  # El número de componentes está en relación al número de filas, en nuestro caso, número de usuarios.
  # además debe ser menor que el número de características (features/usuarios), que son 137:

  # Si viene sin el parámetro, asigne el valor de todos los componentes - 1
  if nComp == None:
    nComp = X.shape[1] - 1
  print("\nComponentes singulares requeridos:", nComp)

  # El número de componentes debe ser menor que el número de características (features/usuarios), que son 943:
  SVD = TruncatedSVD(n_components=nComp, random_state=1)  
  SVD.fit(X)

  num_sv = nComp

  print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
  print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

  # Usemos las primeras {nCompRedu} componentes de dicha factorización.
  # Es decir reducimos la dimensión de la matriz X con solamente sus primeras componentes:
  num_sv = nCompRedu
  SVD = TruncatedSVD(n_components=num_sv, random_state=1)
  resultant_matrix = SVD.fit_transform(X)
  resultant_matrix.shape

  print('\nNúmero de valores singulares:', X.shape[1])
  print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
  print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

  # Obtengamos la matriz de correlación de Pearson entre todas las 
  # variables latentes de los restaurantes:
  corrMx = np.corrcoef(resultant_matrix, )
  corrMx.shape

  # Veamos dónde quedó el índice del restaurante 'Subway' en la matriz de utilidad:
  names = X_P.columns
  names_list = list(names)
  id_megusta = names_list.index(me_gusta)

  # Veamos la correlación deL RESTAURANTE, con todas las que se tienen en el catálogo:
  corr_recomienda = corrMx[id_megusta]
  print('\nTamaño de la matriz de correlación de pearson:', corr_recomienda.shape)
  print('Matriz de correlación de pearson:', corr_recomienda)

  # Listado de los 10 primeros correlaciones
  df_corr=pd.DataFrame(names)
  df_corr['corr']=list(corr_recomienda)
  
  print("\nIndice en el que se ubica [", me_gusta, "] dentro de la matriz: ",  id_megusta)
  print("Lista de los primeros 10 mejores correlacionado con ", me_gusta)
  print(df_corr.sort_values(by='corr', ascending=False).iloc[1:11])


1.1 ..y genera la matriz de utilidad en la cual los renglones son los usuarios
(user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluaciones generales dadas por los usuarios (rating).

In [ ]:
variables_fil = df_geoplaces[['placeID',  'name']]
variables_colu = df_rating[['userID', 'placeID', 'rating']]

1.2 En particular indica el tamaño de dicha matriz y despliega sus primeros 10 renglones. 

In [ ]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd

tabla_relacional = pd.merge(variables_fil , variables_colu , on='placeID')

print(tabla_relacional.shape, "Tamaño de la matriz\n")
print(tabla_relacional.head(10))

(1161, 4) Tamaño de la matriz

   placeID             name userID  rating
0   134999  Kiku Cuernavaca  U1093       2
1   134999  Kiku Cuernavaca  U1066       1
2   134999  Kiku Cuernavaca  U1040       1
3   134999  Kiku Cuernavaca  U1110       2
4   134999  Kiku Cuernavaca  U1121       2
5   132825  puesto de tacos  U1077       2
6   132825  puesto de tacos  U1055       2
7   132825  puesto de tacos  U1053       1
8   132825  puesto de tacos  U1081       1
9   132825  puesto de tacos  U1001       2


#**Ejercicio-2.**

Con el objetivo de obtener la información con respecto a los vectores latentes de los restaurantes, obtener dicha factorización SVD utilizando todas las componentes posibles, es decir, usando todos sus valores singulares. En particular indica la cantidad de valores singulares obtenidos.

In [ ]:
# Ejecutamos la Transpuesta para dejar los restaurantes en las filas.
X_P=tabla_relacional.pivot_table(values='rating', index='userID', columns='name', fill_value=0)
X_P.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
X_P.shape # 129 restaurantes y 138 usuarios

(138, 129)

In [ ]:
import numpy as np

print('Total de elementos de la matriz de utilidad: %d' % (X_P.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(X_P)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(X_P) / X_P.size))

Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%


In [ ]:
# Generamos la transpuesta del PIVOT
X = X_P.T

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Obtengamos la cantidad de información contenida en la factorización SVD, considerando 
# los valores singulares más representativos de dicha factorización.
# Para este caso inicial usaremos todos los vectores singulares, para luego 
# compararlo con el caso al reducir su dimensión.

# El número de componentes está en relación al número de filas, en nuestro caso, número de usuarios.
# además debe ser menor que el número de características (features/usuarios), que son 137:
nComp = X.shape[1] - 1
print("Componentes singulares obtenidos: ", nComp)

# El número de componentes debe ser menor que el número de características (features/usuarios), que son 943:
SVD = TruncatedSVD(n_components=nComp, random_state=1)  
SVD.fit(X)

num_sv = nComp   # En particular usaremos todos los componentes

print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Componentes singulares obtenidos:  137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%


#**Ejercicio-3.**

Usando la métrica de correlación de Pearson, obtener las 10 mejores recomendaciones de este modelo no supervisado con base a la información de alguien que evaluó bien de manera general al “Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10
recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.

In [ ]:
# Usemos las primeras 10 componentes de dicha factorización.
# Es decir reducimos la dimensión de la matriz X con solamente sus primeras componentes:

num_sv = 10
SVD = TruncatedSVD(n_components=num_sv, random_state=1)

resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

(129, 10)

In [ ]:
# Obtengamos la matriz de correlación de Pearson entre todas las 
# variables latentes de los restaurantes:

corrMx = np.corrcoef(resultant_matrix, )
corrMx.shape

(129, 129)

In [ ]:
# Veamos dónde quedó el índice del restaurante 'Subway' en la matriz de utilidad:
me_gusta = "Subway"
names = X_P.columns
names_list = list(names)

id_megusta = names_list.index(me_gusta)
id_megusta

88

In [ ]:
# Veamos la correlación deL RESTAURANTE SUBWAY, con todas las que se tienen en el catálogo:
corr_recomienda = corrMx[id_megusta]
print(corr_recomienda.shape)
print(corr_recomienda)

(129,)
[-0.15064635  0.97877781 -0.1845724  -0.15983695  0.99887223 -0.07193956
 -0.01573204  0.18193487  0.02371689 -0.0721253  -0.12088734  0.99990361
 -0.06747016 -0.12881113 -0.09992082 -0.01851576  0.99669844 -0.05222151
  0.06912151 -0.2865713   0.99326413  0.9972279  -0.02609229 -0.01136617
  0.09097883  0.03500756 -0.06497314 -0.01839213 -0.07070041  0.99712923
 -0.09078034  0.07195591  0.01535248 -0.14201129  0.07756104 -0.17304099
  0.05330264  0.99939672  0.99800799  0.04505749 -0.19152571  0.99990619
  0.99897094 -0.04660154 -0.01223637  0.99987903  0.99997694 -0.16809713
  0.05646127 -0.07411778 -0.15513466  0.99795453 -0.0428533  -0.02754784
  0.99737849  0.99911926 -0.06908106 -0.0535078   0.99995595 -0.37949259
  0.99992724  0.0133778  -0.09546762 -0.04008912  0.9987684  -0.12605574
 -0.07861828  0.02977735 -0.08655958 -0.23038202 -0.23325957  0.0014082
 -0.27000998 -0.09780463  0.05936832 -0.1409088  -0.15376939  0.02895601
  0.03207908 -0.13242192 -0.12764337  0.02587

Veamos qué restaurantes están altamente correlacionadas con el restaurante SUBWAY. 

En particular se omiten las correlaciones iguales a uno o muy cercanas a ellas, la cual en particular estará el restaurante correlacionado consigo mismo. 

Por ejemplo, veamos cuál es el valor de correlación de Pearson del restauante, consigo misma: (Claramente será muy cerca a 1 o 1)

In [ ]:
corr_recomienda[id_megusta]

1.0

3.3 Usando la métrica de correlación de Pearson, obtener las 10 mejores recomendaciones de este modelo no supervisado con base a la información de alguien que evaluó bien de manera general al “Subway”. Despliega también el valor numérico de la correlación de Pearson de cada una de estas 10
recomendaciones. NOTA: No debe estar incluido “Subway” dentro de las recomendaciones.

In [ ]:
df_corr=pd.DataFrame(names)
df_corr['corr']=list(corr_recomienda)
df_corr.sort_values(by='corr', ascending=False).iloc[1:11]

,name,corr
85,Rincon del Bife,0.999991
46,Paniroles,0.999977
58,Restaurant Teely,0.999956
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.999927
41,Mariscos Tia Licha,0.999906
11,Chilis Cuernavaca,0.999904
45,Mikasa,0.999879
97,Vips,0.999704
37,Log Yin,0.999397
86,Sanborns Casa Piedra,0.999228


#**Ejercicio-4.**

Disminuye ahora la dimensión de la factorización con un número de componentes tal, que las recomendaciones con esta menor cantidad de información coincidan en al menos 9 de las 10 recomendaciones dadas con todos los valores singulares. Indica la cantidad de valores singulares encontrado y la cantidad de información que contienen. 

In [ ]:
# EJERCICIO 4, Con: me_gusta='Subway', 'rating'

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil, variables_colu, values='rating', index='userID', columns='name', nComp=137, nCompRedu=137, me_gusta='Subway')

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil, variables_colu, values='rating', index='userID', columns='name', nComp=137, nCompRedu=5, me_gusta='Subway')



######################## Variables recibidas en este ejercicio inicio ##################################
Variables Filas        :  placeID, name
Variables Columnas     :  userID, placeID, rating
Analizando variable    :  rating
Indexado por           :  userID
Columnas               :  name
Total Componentes      :  137
Componentes Reducidos  :  137
Elemento buscado       :  Subway
######################## Variables recibidas en este ejercicio fin ##################################

*** RESULTADOS  

Tamaño de la matriz:  (1161, 4)
Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%

Componentes singulares requeridos: 137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Número de valores singulares: 138
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Tamaño de la matriz de correlación de pearson: (129,)
Matriz de correlaci

#**Ejercicio-5.**

Repite el ejercicio con el restaurante “"tacos de barbacoa enfrente del Tec". Incluye tus conclusiones.

In [ ]:
# EJERCICIO 5, Con: me_gusta='tacos de barbacoa enfrente del Tec', 'rating'

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil, variables_colu, values='rating', index='userID', columns='name', nComp=137, nCompRedu=137, me_gusta='tacos de barbacoa enfrente del Tec')

# Posteriormente ejecutamos y buscamos el mejor valor de componentes reducidos 14, y vemos que obtenemos, la misma información
# que si hubiesemos ejecutado todos los componentes, por supuesto, con la eficiancia de haber obtenido la misma información resultado
# con tan solo 14 componentes y no 137, es decir, una reducción del 89% en la dimensionalidad.
T_SVD(variables_fil, variables_colu, values='rating', index='userID', columns='name', nComp=137, nCompRedu=14, me_gusta='tacos de barbacoa enfrente del Tec')



######################## Variables recibidas en este ejercicio inicio ##################################
Variables Filas        :  placeID, name
Variables Columnas     :  userID, placeID, rating
Analizando variable    :  rating
Indexado por           :  userID
Columnas               :  name
Total Componentes      :  137
Componentes Reducidos  :  137
Elemento buscado       :  tacos de barbacoa enfrente del Tec
######################## Variables recibidas en este ejercicio fin ##################################

*** RESULTADOS  

Tamaño de la matriz:  (1161, 4)
Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 907
Porcentaje de elementos diferentes de cero: 5.1%

Componentes singulares requeridos: 137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Número de valores singulares: 138
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Tamaño de la matriz de correlación de pearson

Se enlistan como recomendación los 10 restaurantes que tienen mayor correlación con el restaurante "tacos de barbacoa enfrente del Tec", lo cual indica que los vectores que los representan son muy parecidos entre sí. Nos damos cuenta que la correlación es muy alta en los 10 casos enlistados, con un índice mayor al 99%, por lo que prácticamente no existe diferencia entre ellos lo cual indica que cualquiera de estas recomendaciones gustará por igual a los usuarios.

#**Ejercicio-6.**

In [ ]:
# Nuevas variables, ahora analizando con FOOT_RATING.
variables_fil_2= df_geoplaces[['placeID', 'name']]
variables_col_2 = df_rating[['userID', 'placeID', 'food_rating']]

# EJERCICIO 6, Con: me_gusta='Subway', 'food_rating'

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_2, variables_col_2, values='food_rating', index='userID', columns='name', nComp=137, nCompRedu=137, me_gusta='Subway')

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_2, variables_col_2, values='food_rating', index='userID', columns='name', nComp=137, nCompRedu=72, me_gusta='Subway')



######################## Variables recibidas en este ejercicio inicio ##################################
Variables Filas        :  placeID, name
Variables Columnas     :  userID, placeID, food_rating
Analizando variable    :  food_rating
Indexado por           :  userID
Columnas               :  name
Total Componentes      :  137
Componentes Reducidos  :  137
Elemento buscado       :  Subway
######################## Variables recibidas en este ejercicio fin ##################################

*** RESULTADOS  

Tamaño de la matriz:  (1161, 4)
Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 895
Porcentaje de elementos diferentes de cero: 5.0%

Componentes singulares requeridos: 137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Número de valores singulares: 138
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Tamaño de la matriz de correlación de pearson: (129,)
Matriz de

Al realizar la evaluación por la calificación de la comida, 'food_rating', y con un universo reducido a 137 elementos notamos que de las 10 recomendaciones con mayor correlación, solo la primera presenta una tendencia significativa del 81.64%, las demás están por debajo del 50% lo cual indicaría que las 9 restantes tienden a una indiferencia por la poca correlación existente con el restaurante Subway.
Al reducir el universo a 72 vectores singulares solamente, vemos que la correlación mayor es del 88.12%, siendo la única positiva con tendencia a coincidir con el restaurante Subway. Existe solamente una intermedia con el 51% y las demás tienden a ser indiferentes por tener una correlación que disminuye por debajo del 50%.
Por medio de esta reducción se optimizó el modelo brindando información más precisa que permite una mejor visualización predictiva de las preferencias de restaurantes.

In [ ]:
# EJERCICIO 6, Con: me_gusta='tacos de barbacoa enfrente del Tec', 'food_rating'

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_2, variables_col_2, values='food_rating', index='userID', columns='name', nComp=137, nCompRedu=137, me_gusta='tacos de barbacoa enfrente del Tec')

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_2, variables_col_2, values='food_rating', index='userID', columns='name', nComp=137, nCompRedu=10, me_gusta='tacos de barbacoa enfrente del Tec')



######################## Variables recibidas en este ejercicio inicio ##################################
Variables Filas        :  placeID, name
Variables Columnas     :  userID, placeID, food_rating
Analizando variable    :  food_rating
Indexado por           :  userID
Columnas               :  name
Total Componentes      :  137
Componentes Reducidos  :  137
Elemento buscado       :  tacos de barbacoa enfrente del Tec
######################## Variables recibidas en este ejercicio fin ##################################

*** RESULTADOS  

Tamaño de la matriz:  (1161, 4)
Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 895
Porcentaje de elementos diferentes de cero: 5.0%

Componentes singulares requeridos: 137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Número de valores singulares: 138
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Tamaño de la matriz de correlación 

Al realizar el ejercicio con el restaurante "tacos de barbacoa enfrente del Tec" y basándonos en la calificación de la comida utilizando una muestra de 137 vectores singulares, vemos cómo solamente 5 restaurantes son los que presentan una correlación con tendencia a gustar a los usuarios, los otros 5 tienen una correlación menor al 50%, indicando que se alejan de la preferencia de los usuarios.
Por otra parte, realizando nuevamente el ejercicio con solamente 10 vectores singulares notamos como la correlación aumenta de manera significativa a más del 99% en las 10 recomendaciones, lo cual presupone que al tener un menor volumen de vectores singulares existe una menor dispersión entre ellos, por lo que la correlación aumenta significativamente.

#**Ejercicio-7.**

In [ ]:
variables_fil_3= df_geoplaces[['placeID', 'name']]
variables_col_3 = df_rating[['userID', 'placeID', 'service_rating']]

# EJERCICIO 7, Con: me_gusta='Subway', 'service_rating'

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_3, variables_col_3, values='service_rating', index='userID', columns='name', nComp=137, nCompRedu=137, me_gusta='Subway')

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_3, variables_col_3, values='service_rating', index='userID', columns='name', nComp=137, nCompRedu=47, me_gusta='Subway')



######################## Variables recibidas en este ejercicio inicio ##################################
Variables Filas        :  placeID, name
Variables Columnas     :  userID, placeID, service_rating
Analizando variable    :  service_rating
Indexado por           :  userID
Columnas               :  name
Total Componentes      :  137
Componentes Reducidos  :  137
Elemento buscado       :  Subway
######################## Variables recibidas en este ejercicio fin ##################################

*** RESULTADOS  

Tamaño de la matriz:  (1161, 4)
Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 846
Porcentaje de elementos diferentes de cero: 4.8%

Componentes singulares requeridos: 137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Número de valores singulares: 138
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Tamaño de la matriz de correlación de pearson: (129,)
Mat

Al realizar el ejercicio utilizando como referencia la calificación del servicio, service_rating, y con la muestra completa de 137 vectores singulares nos damos cuenta que existe una dispersión importante que no refleja una alta correlación, lo que hace que la más alta sea del 77.29% y presente solamente 3 recomendaciones medianamente similares a la del restaurante SUBWAY. Posteriormente, al optimizar el modelo reduciendo el número de vectores singulares a 47, encontramos que el modelo nos arroja un valor máximo del 95.75% acompañado de otras 3 recomendaciones por encima del 50%, las otras 5 tienden a tener una menor correlación.

In [ ]:
# EJERCICIO 7, Con: me_gusta='tacos de barbacoa enfrente del Tec', 'service_rating'

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_3, variables_col_3, values='service_rating', index='userID', columns='name', nComp=137, nCompRedu=137, me_gusta='tacos de barbacoa enfrente del Tec')

# Ejecutamos siempre con el total de componentes nCompRedu y verificamos los 10 primeros correlacionados
T_SVD(variables_fil_3, variables_col_3, values='service_rating', index='userID', columns='name', nComp=137, nCompRedu=5, me_gusta='tacos de barbacoa enfrente del Tec')



######################## Variables recibidas en este ejercicio inicio ##################################
Variables Filas        :  placeID, name
Variables Columnas     :  userID, placeID, service_rating
Analizando variable    :  service_rating
Indexado por           :  userID
Columnas               :  name
Total Componentes      :  137
Componentes Reducidos  :  137
Elemento buscado       :  tacos de barbacoa enfrente del Tec
######################## Variables recibidas en este ejercicio fin ##################################

*** RESULTADOS  

Tamaño de la matriz:  (1161, 4)
Total de elementos de la matriz de utilidad: 17802
Total de elementos diferentes de cero: 846
Porcentaje de elementos diferentes de cero: 4.8%

Componentes singulares requeridos: 137
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Número de valores singulares: 138
Cantidad de información simplificada con los primeros 137 vectores singulares:
0.0%

Tamaño de la matriz de correl

Realizando ahora el ejercicio valorando la columna service_rate para el restaurante "tacos de barbacoa enfrente del Tec" nos pudimos percatar que al utilizar una muestra de 137 vectores singulares obtuvimos solamente 3 recomendaciones afines al restaurante, presentando la mayor una correlación del 91.23%, mientras que las otras 2 cuentan con una correlación medianamente positiva del 58% y 51%.
Al llevar a cabo la optimización mediante la reducción de los vectores singulares a solamente 5, obtuvimos las 10 recomendaciones en un rango del 98% al 99%, por lo que concluimos al igual que en los ejercicios anteriores, que la reducción por SVD nos permite tener una mayor correlación al ser menor la dispersión entre menos elementos considere el modelo.

#**Ejercicio-8.**

Al llevar a cabo la comparativa de recomendaciones entre los 3 modelos notamos 3 consideraciones importantes:
1. Las recomendaciones cambian en función de la informacón referente a cada columna que se evalúa, por lo que no son exactamente los mismos resultados entre los 3 modelos.
2. Cada vez que utilizamos la Reducción de Dimensionalidad por Descomposición en Valores Singulares (SVD), el modelo se optimiza mostrando de manera más eficiente mejores correlaciones entre los vectores. Como ya se había mencionado, esto indica que ante un número reducido de vectores singulares la dispersión es menor, y por ende, las correlaciones son mayores, lo cual nos indica de forma más acertiva las recomendaciones que son mejores.
3. Al llevar a cabo la Reducción por Dimensionalidad las recomendaciones llegaron a cambiar en un 10%, es decir, una de las 10 recomendaciones obtenidas al utilizar el volumen completo de vectores invariablemente cambió, independientemente de que todas mostraron también correlaciones más altas.
Considerando lo anterior, el proceso de Descomposición en Valores Singulares (SVD) representa una forma de optimizar los modelos de recomendación, por medio del cual se pueden obtener correlaciones más precisas y de forma más ágil de la data que se analice.

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**